In [7]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
import os

/Users/amber/Documents/Learnings_2024/Langraph_2025/langraph_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Load environment variables from the .env file
load_dotenv()

True

In [9]:

# # Access the variables
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [10]:
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash',google_api_key=gemini_api_key)

In [11]:

class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [12]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [13]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [14]:

graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [15]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!",
 'explanation': 'This is a pun, relying on the similar sound of "fungi" (the plural of fungus, a type of mushroom) and "fun guy".  The joke plays on the double meaning of "fungi" to create a humorous and unexpected connection between a pizza slice and a mushroom.  The pizza slice\'s liking of the mushroom is a playful pretext to deliver the pun.'}

In [16]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explanation': 'This is a pun, relying on the similar sound of "fungi" (the plural of fungus, a type of mushroom) and "fun guy".  The joke plays on the double meaning of "fungi" to create a humorous and unexpected connection between a pizza slice and a mushroom.  The pizza slice\'s liking of the mushroom is a playful pretext to deliver the pun.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074103-1a7d-6102-8002-f927965bc002'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-08T04:28:54.986557+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074103-0b5d-660e-8001-91249c6c6983'}}, tasks=(), interrupts=())

In [17]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explanation': 'This is a pun, relying on the similar sound of "fungi" (the plural of fungus, a type of mushroom) and "fun guy".  The joke plays on the double meaning of "fungi" to create a humorous and unexpected connection between a pizza slice and a mushroom.  The pizza slice\'s liking of the mushroom is a playful pretext to deliver the pun.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074103-1a7d-6102-8002-f927965bc002'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-08T04:28:54.986557+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074103-0b5d-660e-8001-91249c6c6983'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!"}, n

In [18]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': "Why did the Italian chef quit his job?  Because he didn't get enough *pasta*bilities!",
 'explanation': 'The joke plays on the double meaning of "pasta-bilities."\n\n* **Pasta:**  Refers to the Italian food.\n* **Possibilities:** Refers to opportunities or chances.\n\nThe chef quit because he felt his job didn\'t offer him enough opportunities for growth, creativity, or advancement.  The pun uses the similar-sounding word "pasta" to create a humorous and unexpected twist.'}

In [19]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explanation': 'This is a pun, relying on the similar sound of "fungi" (the plural of fungus, a type of mushroom) and "fun guy".  The joke plays on the double meaning of "fungi" to create a humorous and unexpected connection between a pizza slice and a mushroom.  The pizza slice\'s liking of the mushroom is a playful pretext to deliver the pun.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074103-1a7d-6102-8002-f927965bc002'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-08T04:28:54.986557+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074103-0b5d-660e-8001-91249c6c6983'}}, tasks=(), interrupts=())

In [20]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explanation': 'This is a pun, relying on the similar sound of "fungi" (the plural of fungus, a type of mushroom) and "fun guy".  The joke plays on the double meaning of "fungi" to create a humorous and unexpected connection between a pizza slice and a mushroom.  The pizza slice\'s liking of the mushroom is a playful pretext to deliver the pun.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074103-1a7d-6102-8002-f927965bc002'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-08T04:28:54.986557+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074103-0b5d-660e-8001-91249c6c6983'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!"}, n

### Time Travel

In [22]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f074103-04ba-6fcc-8000-b724dbd58cc2"}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f074103-04ba-6fcc-8000-b724dbd58cc2'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-08-08T04:28:52.705051+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074103-04b2-64f8-bfff-3ca79d41cd85'}}, tasks=(PregelTask(id='bea0dc87-ca13-5a69-f1f7-d97be860f8de', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!"}),), interrupts=())

In [23]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f074103-04ba-6fcc-8000-b724dbd58cc2"}})

{'topic': 'pizza',
 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!",
 'explanation': 'This is a pun, a joke that relies on the similar sound of two words with different meanings.\n\n* **Fungi:** This refers to a kingdom of organisms including mushrooms, molds, and yeasts.\n\n* **Fun guy:** This is a playful phrase meaning a fun person.\n\nThe joke plays on the similar pronunciation of "fungi" and "fun guy."  The pizza slice liking the mushroom is a silly premise, but the punchline uses the word "fungi" to create a humorous association with "fun guy," suggesting the pizza slice enjoys the mushroom\'s company because the mushroom is a fun companion.'}

In [24]:

list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explanation': 'This is a pun, a joke that relies on the similar sound of two words with different meanings.\n\n* **Fungi:** This refers to a kingdom of organisms including mushrooms, molds, and yeasts.\n\n* **Fun guy:** This is a playful phrase meaning a fun person.\n\nThe joke plays on the similar pronunciation of "fungi" and "fun guy."  The pizza slice liking the mushroom is a silly premise, but the punchline uses the word "fungi" to create a humorous association with "fun guy," suggesting the pizza slice enjoys the mushroom\'s company because the mushroom is a fun companion.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074122-43a4-6160-8002-003cd895427e'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-08T04:42:51.451619+00:00', parent_config={'configurable': {'thread_id': '1'

### Updating state

In [25]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f074103-04ba-6fcc-8000-b724dbd58cc2", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f074128-9270-6446-8001-b44aecbef41d'}}

In [26]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074128-9270-6446-8001-b44aecbef41d'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-08-08T04:45:40.775413+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074103-04ba-6fcc-8000-b724dbd58cc2'}}, tasks=(PregelTask(id='fe6f5bd9-b8f9-48d6-9b0f-6211c5832d11', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explanation': 'This is a pun, a joke that relies on the similar sound of two words with different meanings.\n\n* **Fungi:** This refers to a kingdom of organisms including mushrooms, molds, and yeasts.\n\n* **Fun guy:** This is a playful phrase meani

In [27]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f074128-9270-6446-8001-b44aecbef41d"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa get a bad review?  Because it was a little *pastry*.',
 'explanation': 'The joke plays on the double meaning of "pastry."\n\n* **Pastry (meaning 1):**  Refers to a type of baked or fried dough, which is what a samosa is.  This is the literal meaning in the context of the joke\'s setup.\n\n* **Pastry (meaning 2):**  Refers to something that\'s past its prime, outdated, or stale.  This is the punchline\'s meaning.\n\nThe humor comes from the unexpected shift in meaning. The listener initially understands "pastry" in the culinary sense, relating to the samosa\'s composition.  The punchline then subverts this expectation by using the word to imply the samosa was old and therefore of poor quality, hence the "bad review."  It\'s a simple pun relying on wordplay.'}

In [28]:

list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa get a bad review?  Because it was a little *pastry*.', 'explanation': 'The joke plays on the double meaning of "pastry."\n\n* **Pastry (meaning 1):**  Refers to a type of baked or fried dough, which is what a samosa is.  This is the literal meaning in the context of the joke\'s setup.\n\n* **Pastry (meaning 2):**  Refers to something that\'s past its prime, outdated, or stale.  This is the punchline\'s meaning.\n\nThe humor comes from the unexpected shift in meaning. The listener initially understands "pastry" in the culinary sense, relating to the samosa\'s composition.  The punchline then subverts this expectation by using the word to imply the samosa was old and therefore of poor quality, hence the "bad review."  It\'s a simple pun relying on wordplay.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07412a-06f4-6e06-8003-588796d9c8d0'}}, metadata={'source': 'loop', '

### Fault Tolerance